In [1]:
import pandas as pd
import time
from io_data.dates import Dates_Operations
from ml_models.preprocessing import Preprocessing
from ml_models.groups import GROUPS
from ml_models.postprocessing import Postprocessing
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
logging.getLogger("cmdstanpy").propagate = False
logging.getLogger("cmdstanpy").disabled = True
import matplotlib.pyplot as plt
plt.ioff()

prefix = 'D:/работа/groups/data/'
prefix_plots = 'D:/работа/groups/TEST_PLOTS/'
prefix_output = 'D:/работа/groups/RESULTS_NEW/'

Importing plotly failed. Interactive plots will not work.


## Чтение файла с исходными фактическими данными

In [2]:
df = Preprocessing.get_data_for_forecast(filename = f'{prefix}Data_by_months_TEST.xlsx',
                  list_of_replacements = ['All 18+', 'All 14-59', 'All 10-45', 'All 14-44',
                      'All 14-54', 'All 25-49', 'All 25-54', 'All 4-45', 'All 6-54', 'W 14-44', 'W 25-59'], 
                                         column_name_with_date = 'Date')

## Определение количества месяцев для прогноза

In [3]:
start_date = pd.to_datetime(df.index[-1], format = '%d.%m.%Y') #нахождение последнего месяца в DataFrame
stop_date = '31.12.2025' # 31 декабря текущего года
forecast_periods = Dates_Operations.make_difference_in_months(start_date, stop_date)
forecast_periods

12

## Функция для расчета

In [4]:
def main(df,
         forecast_periods,
         column_name_with_date, 
         weights_filepath, 
         error_dir = None,
         plots_dir = None, 
         save_dir = None, 
         plots: bool = False, 
         test: bool = False):
    """
        Функция для запуска прогноза ансамбля ML-моделей
        Args:
            filename: Полный путь к файлу с исходными данными
            list_of_replacements: Список из листов, находящихся в файле с исходными данными
            column_name_with_date: Название столбца с датой
            weights_filepath: Полный путь к файлу с весами для каждой модели
            plots_dir: Путь к директории, куда будут сохраняться графики для каждой модели
            save_dir: Путь к директории, куда будут сохраняться итоговые графики с прогнозами
            plots: Переменная типа bool. Если True, то графики строятся. В противном случае нет.
            test: Переменная типа bool. Если True, то тестинг проводится. В противном случае нет.
        Returns:
    """
    #Определение к какой группе относятся данные по тому или иному каналу
    group_1, group_2, group_3, group_4 = GROUPS(df).initiate_group()

    avg_forecasts = []
    
    #GROUP_1
    if not group_1.empty:
        print('', 'Результаты работы различных методов для ТВ-каналов с сезонностью и трендом', sep = '\n', end = '\n')
        avg_forecast_1 = GROUPS(group_1).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_1', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print('== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ С СЕЗОННОСТЬЮ И ТРЕНДОМ ==', avg_forecast_1, sep = '\n', end = '\n')
        if plots:
            Postprocessing(group_1, avg_forecast_1).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Cезонность и тренд')
        avg_forecasts.append(avg_forecast_1)
        
    #GROUP_2
    if not group_2.empty:
        print('', 'Результаты работы различных методов для ТВ-каналов с трендом без сезонности', sep = '\n', end = '\n')
        avg_forecast_2 = GROUPS(group_2).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_2', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print('== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ С ТРЕНДОМ БЕЗ СЕЗОННОСТИ ==', avg_forecast_2, sep = '\n', end = '\n')
        if plots:
            Postprocessing(group_2, avg_forecast_2).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Тренд без сезонности')
        avg_forecasts.append(avg_forecast_2)

    #GROUP_3
    if not group_3.empty:
        print('', 'Результаты работы различных методов для ТВ-каналов с сезонностью без тренда', sep = '\n', end = '\n')
        avg_forecast_3 = GROUPS(group_3).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_3', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print('== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ С СЕЗОННОСТЬЮ БЕЗ ТРЕНДА ==', avg_forecast_3, sep = '\n', end = '\n')
        if plots:
            Postprocessing(group_3, avg_forecast_3).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Сезонность без тренда')
        avg_forecasts.append(avg_forecast_3)

    #GROUP_4
    if not group_4.empty:
        print('', 'Результаты работы различных методов для ТВ-каналов без сезонности и без тренда', sep = '\n', end = '\n')
        avg_forecast_4 = GROUPS(group_4).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_4', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print('== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ БЕЗ СЕЗОННОСТИ И БЕЗ ТРЕНДА ==', avg_forecast_4, sep = '\n', end ='\n')
        if plots:
            Postprocessing(group_4, avg_forecast_4).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Без сезонности и без тренда')
        avg_forecasts.append(avg_forecast_4)
    general_df = Postprocessing.ensemble_of_models(df, *avg_forecasts)
    return general_df, test

# **Расчет**

In [5]:
start = time.perf_counter()

general_df, test = main(df = df,
                  forecast_periods = forecast_periods, 
                  column_name_with_date = 'Date', 
                  weights_filepath = 'ml_models/config.json', 
                  error_dir = 'D:/работа/groups/ERRORS',
                  plots_dir = 'D:/работа/groups/PLOTS_NEW/',
                  save_dir = 'D:/работа/groups/RESULT_NEW/',
                  plots = True, 
                  test = True)

end = time.perf_counter()

print(f'ВРЕМЯ РАБОТЫ КОДА СОСТАВИЛО: {((end-start) / 60):0.2f} мин.')


Результаты работы различных методов для ТВ-каналов с трендом без сезонности
РЕЗУЛЬТАТ РАБОТЫ ФУНКЦИИ REGR_LIN
РЕЗУЛЬТАТ РАБОТЫ ФУНКЦИИ SEASONDEC_PERIODS
РЕЗУЛЬТАТ РАБОТЫ ФУНКЦИИ NAIVE_WITH_ERROR
РЕЗУЛЬТАТ РАБОТЫ ФУНКЦИИ NAIVE_WITH_ERROR_LAST_6_MONTHS
            ДОМАШНИЙ (БАРНАУЛ) All 18+  ЗВЕЗДА (БАРНАУЛ) All 18+  \
Date                                                               
2024-01-01                      6.9060                    2.8551   
2024-02-01                      6.8940                    2.9369   
2024-03-01                      6.9203                    2.6247   
2024-04-01                      7.5167                    3.1095   
2024-05-01                      6.1124                    3.0924   
2024-06-01                      7.3289                    3.2770   
2024-07-01                      7.4513                    3.6693   
2024-08-01                      8.5544                    2.6694   
2024-09-01                      7.5642                    2.6119   


In [6]:
general_df

,ДОМАШНИЙ (БАРНАУЛ) All 18+,ЗВЕЗДА (БАРНАУЛ) All 18+,НТВ (БАРНАУЛ) All 18+,ПЕРВЫЙ КАНАЛ (БАРНАУЛ) All 18+,ПЯТЫЙ КАНАЛ (БАРНАУЛ) All 18+,РЕН ТВ (БАРНАУЛ) All 18+,РОССИЯ 1 (БАРНАУЛ) All 18+,РОССИЯ 24 (БАРНАУЛ) All 18+,ТВ ЦЕНТР (БАРНАУЛ) All 18+,ТВ-3 (БАРНАУЛ) All 18+,...,ПЯТНИЦА (ЕКАТЕРИНБУРГ) All 14-44,ТНТ (ЕКАТЕРИНБУРГ) All 14-44,ПЯТНИЦА (БАРНАУЛ) All 14-54,ТНТ (ЧЕЛЯБИНСК) All 14-54,ЧЕ (САНКТ-ПЕТЕРБУРГ) All 25-49,ТВ-3 (ЕКАТЕРИНБУРГ) All 25-54,КАРУСЕЛЬ (ВОЛГОГРАД) All 4-45,СТС (БАРНАУЛ) All 6-54,Ю (ЕКАТЕРИНБУРГ) W 14-44,ДОМАШНИЙ (ЕКАТЕРИНБУРГ) W 25-59
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-01,6.5636,2.6848,9.8851,9.3688,4.7076,6.3612,14.9089,1.8119,3.0388,2.7331,...,6.2152,9.7639,3.5476,9.0123,1.1900,1.5959,11.0669,8.5338,2.1493,2.4750
2024-02-01,6.6516,2.6304,9.9249,10.0393,5.0228,6.7204,16.4319,1.7797,2.9668,2.5948,...,6.4558,10.7479,3.5078,9.4523,0.7918,1.8198,12.1225,7.9507,2.5829,2.3548
2024-03-01,6.4716,2.4809,9.4818,10.7993,5.0132,6.8973,15.6641,2.4926,3.4871,2.7363,...,6.5150,9.9182,3.4067,9.0836,1.0917,1.6271,12.2122,9.3392,2.0695,1.9196
2024-04-01,6.7798,2.8436,9.6645,10.0260,5.5547,6.6703,16.5541,2.0586,3.6148,2.5603,...,5.9293,9.9040,3.5449,8.7516,1.0520,1.8118,12.7364,8.0470,1.8972,2.5111
2024-05-01,5.9226,2.7742,9.4015,9.4148,5.3791,6.4046,16.0422,2.1426,3.7508,2.6319,...,5.9446,9.7765,4.0442,7.5751,0.8744,1.9602,12.6387,8.6937,1.2659,2.3757
2024-06-01,6.9958,2.8497,9.2720,9.6432,5.3147,6.7677,15.9163,2.0783,3.9827,2.6441,...,5.7279,10.0465,4.0288,7.9528,0.8577,1.7021,13.9252,8.0799,1.9531,1.7066
2024-07-01,7.0168,3.1489,10.0086,9.3122,5.3541,6.4400,15.4756,1.8177,3.9157,3.4747,...,5.9951,10.2819,4.1007,8.3071,0.8029,1.9555,14.5211,7.9572,1.9726,1.7693
2024-08-01,7.6065,2.4305,9.5341,9.6781,5.4114,6.8237,14.9199,2.1382,3.0981,2.9310,...,5.5536,9.7275,4.7863,8.6244,0.7249,1.6720,13.1370,7.5985,2.4864,1.8018
2024-09-01,6.6943,2.4900,8.7912,10.7912,4.9042,6.5897,16.8498,1.7151,3.1815,2.5294,...,5.7937,10.2526,4.7325,8.5747,0.8284,1.8570,12.9709,8.0716,3.0306,1.9104


# **Вывод ошибок прогноза**

In [7]:
if test and forecast_periods <= 12:
    test_data =df.iloc[-forecast_periods:]
    general_MAPE = (general_df/test_data - 1) * 100
    general_MAPE = general_MAPE.reset_index()
    mean_values = general_MAPE.mean()
    # Добавляем новую строку с названием индекса "Mean"
    general_MAPE.loc['Mean'] = mean_values
    general_MAPE.to_excel(f'{prefix_output}forecast_error.xlsx')

In [8]:
general_df = general_df.reset_index()
general_df.to_excel(f'{prefix_output}forecast_output.xlsx')